In [1]:
!pip install bigdl-llm[all]

# Additional package required for Baichuan-13B-Chat to conduct generation
!pip install -U transformers_stream_generator


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/3.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.7 MB 960.0 kB/s eta 0:00:04
     -- ------------------------------------- 0.2/3.7 MB 2.3 MB/s eta 0:00:02
     ---- ----------------------------------- 0.4/3.7 MB 3.4 MB/s eta 0:00:01
     ------ --------------------------------- 0.6/3.7 MB 4.2 MB/s eta 0:00:01
     --------- ------------------------------ 0.9/3.7 MB 4.7 MB/s eta 0:00:01
     ------------ --------------------------- 1.2/3.7 MB 4.9 MB/s eta 0:00:01
     --------------- ------------------------ 1.4/3.7 MB 5.3 MB/s eta 0:00:01
     ------------------ --------------------- 1.8/3.7 MB 5.6 MB/s eta 0:00:01
     --------------------- ------------------ 2.0/3.7 MB 5.7 MB/s eta 0:00:01
     ----------------------- ---------------- 2.2/3.7 MB 5.5 MB/s eta 0:00:01
     ------------------------ --------------- 2.3/3.7 MB 5.4 MB/s eta 0:00:01
     ---

In [2]:
from bigdl.llm.transformers import AutoModelForCausalLM

model_path = "C:\LLM\llm-model\saved-int4-models\Baichuan-13B-Chat"
#model = AutoModelForCausalLM.from_pretrained(model_path,
#                                             load_in_4bit=True,
#                                             trust_remote_code=True)
model = AutoModelForCausalLM.load_low_bit(model_path,trust_remote_code=True)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path,
                                          trust_remote_code=True)

c:\Users\86177\anaconda3\envs\vllm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers.generation.utils import GenerationConfig
model.generation_config = GenerationConfig.from_pretrained("C:\LLM\llm-model\saved-int4-models\Baichuan-13B-Chat")

In [20]:
def multistep_prompt_zjh_1(user_input):
    return f"""您是一个大型语言模型提示词撰写专家。
        用户的输入可能涉及一个复杂的问题，如果直接让语言模型回答这个问题效果并不好。你的目的是将用户的问题拆分为几个小问题，让语言模型一步步思考，逐渐靠近答案。优化过的问题将以“请逐个回答上述问题，一步步思考。”结尾。

        例子：
        输入：
        我希望按欧式风格装修房子，要花费多少钱？

        修改后的输入：
        1.AI助手你好，欧式风格的房子主要有哪些家具？\n2.AI助手你好，欧式风格的家具一般需要多少钱？\n3.AI助手你好，完成欧式风格的装修总共需要多少钱？\n请逐个回答上述问题，一步步思考。

        原来的输入如下：
        {user_input}

        修改的输入如下：
        """

In [21]:

def multistep_prompt_prompter(question):
    messages = []
    messages.append({"role":"user","content":multistep_prompt_zjh_1(question)})
    response = model.chat(tokenizer, messages)
    print(response)

### feature extraction

In [62]:
def feature_extraction_prompt_zjh_1(user_history):
    return f"""您是独属于该用户的提示词管家。
        您会定期看到用户向大型语言模型提问的历史记录，你需要根据这些历史提问，提取出几个用户的特征，包括但不限于使用语言、倾向于提出的问题类型、提问的立场、用户可能的身份、经常犯的逻辑谬误等等。
        我们会将这些特征加入到大型语言模型的prompt中，以便于模型更好地理解用户的提问。
        你需要在每个特征前面加上#号，以便于我们识别。

        例子1：
        历史记录：
        r如何将数组变成字符串
        enumerate放回参数
        find如何能把div下面包的所有内容都找到
        while至多3次怎么写
        我有一个网页请求经常会失败，导致程序停止，我如何让它失败时再来两遍
        
        提取出的特征：
        #用户经常提出编程问题。\n#用户经常提出关于Python的问题。

        例子2：
        历史记录：
        去北京旅游有什么交通出行的选择吗
        北京有什么好玩的地方吗
        北京有什么好吃的吗
        北京有什么好玩的吗
        北京著名景点有哪些

        提取出的特征：
        #用户喜欢旅游\n#用户计划去北京。

        以上例子以及特征与你所服务的客户无关，仅供参考。

        历史记录：
        {user_history}

        提取出的特征：
        """

In [80]:
feature=[]

In [81]:

import re
def feature_extraction_prompt_prompter(history):
    messages = []
    messages.append({"role":"user","content":feature_extraction_prompt_zjh_1(history)})
    response = model.chat(tokenizer, messages)
    # print(response)
    pattern = r"#(.*)"
    matches = re.findall(pattern, response)
    # print(matches)
    return matches

In [82]:
history = '''
Line 1053: Char 9: runtime error: addition of unsigned offset to 0x7ffd970ec650 overflowed to 0x7ffd970ec64f (basic_string.h)
SUMMARY: UndefinedBehaviorSanitizer: undefined-behavior /usr/bin/../lib/gcc/x86_64-linux-gnu/11/../../../../include/c++/11/bits/basic_string.h:1062:9这个怎么回事
html文件如何用python改成{"titile":"xxx","content":"xxx"}
llm中strong reasoner什么意思
instruction tuning改变参数吗
'''
match=feature_extraction_prompt_prompter(history)
feature.append(match[0])
# 也可以使用extend来将所有特征加上去，或者加一个也可以（使用append）

In [83]:
history = '''
报考高校需要注意什么
志愿填报分几个阶段
土木工程什么学校比较好
'''

match=feature_extraction_prompt_prompter(history)
feature.append(match[0])
print(feature)

['用户对编程和计算机相关知识感兴趣。', '用户可能正在准备高考或大学入学考试。']


### multistep

In [22]:
question="我想准备一顿烛光晚餐，需要花费多少钱？"
multistep_prompt_prompter(question)

请分以下步骤回答上述问题：

1.首先确定你想要提供的 afp 的种类和数量。例如，你可以提供三道不同的菜肴，每道价格为 $5，那么总费用就是 $5 x 3 = $15。


### default

In [53]:
question1="复旦 张维维"
prompt_prompter(question1)


你好，chatgpt，我想向你咨询一下问题：在复旦大学中，张维维教授的研究方向和主要成就有哪些？


In [54]:
question2="中国现在低位"
prompt_prompter(question2)

$你好，chatҐPT，我想向你咨询一下问题：中国现在的经济地位如何？请详细解释并举例说明。


In [55]:
question3="请为我介绍维达定理"
prompt_prompter(question3)

你好，chatgpt，我想向你咨询一下问题：你能详细解释一下维达定理吗？谢谢！


In [56]:
question4="复旦和交大选哪个"
prompt_prompter(question4)

你好，chatgpt，我想向你咨询一下问题：如果我要选择一所中国顶尖的大学进行学习，是应该选择复旦大学还是上海交通大学呢？


In [57]:
question5="什么是数学归纳法，我不懂"
prompt_prompter(question5)

$你好，chatgpt，我想向你咨询一下问题：能否简单解释一下什么叫做数学归纳法？谢谢！


In [59]:
question6="我想和女朋友去北京玩，有攻略吗"
prompt_prompter(question6)

你好，chatgpt，我想向你咨询一下问题：如何规划一次北京之行以便我和我的女朋友共同度过一段愉快的时光？


In [60]:
question7="大学生早上起不来咋办"
prompt_prompter(question7)

你好，chatgpt，我想向你咨询一下问题：如果你是一名大学生，早晨总是难以按时起床，有哪些策略或者方法可以帮助你改善这个情况呢？
